In [2]:
from braket.aws import AwsDevice
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.config import InstanceConfig

import boto3

import os
import json
import time
import altair as alt
import pandas as pd
import numpy as np

In [11]:
import copy

c = copy.deepcopy

def obtain_default_bucket(target: str) -> str:
    with open(os.path.abspath('../..') + '/.default-setting') as f:
        lines = f.readlines()
        for line in lines:
            if (line.startswith(target+'=')):
                return line.split('=')[1].strip()

def get_key(single_dict):
    for k in single_dict.keys():
        return k

def parse_params(params_list, hp, hp_list):
    params = params_list[0]
    k = get_key(params)
    ps = params[k]
    for p in ps:
        hp[k] = p
        if len(params_list) == 1:
            hp_list.append(c(hp))
        else:
            parse_params(params_list[1:], hp, hp_list)

def get_quantum_device(device_name):
    device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
    try:
        device = AwsDevice.get_devices(names=[device_name])
        device_arn = device[0].arn
    except Exception as e:
        print(f"fail to get {device_name}: {e}, use sv1 instead")
    return device_arn

def upload_data(dir_name, aws_session=AwsSession()):
    stream_s3_uri = aws_session.construct_s3_uri(obtain_default_bucket("bucketName"), dir_name)
    return_path = None
    
    def _check_upload(file_name, check_list):
        file_end = file_name.split('.')[-1]
        if file_end in check_list:
            path = f"{stream_s3_uri}/" + file_name.split('/')[-1]
            aws_session.upload_to_s3(file_name, path)
            
    if os.path.isdir(dir_name):
        dir_list = os.listdir(dir_name)
        for file_name in dir_list:
            _check_upload(os.path.join(dir_name,file_name), ['mol2'])
        return_path = stream_s3_uri
    else:
        _check_upload(file_name, ['mol2'])
        single_file_name = file_name.split('/')[-1]
        return_path = f"{stream_s3_uri}/{single_file_name}"
        
    return return_path

def queue_check(jobs):
    queue_count = 0
    running_count = 0
    check_pass = True
    for job in jobs:
        # print(f"job state {job.state()}")
        if job.state() == "QUEUED":
            queue_count = queue_count + 1
        if job.state() == "RUNNING":
            running_count = running_count + 1
        if queue_count + running_count >= 5:
            check_pass = False
    
    print(f"queue_count {queue_count}, running_count {running_count}")
    
    return check_pass

def get_result(result):
    return [result["hypermeter"]["D"], result["hypermeter"]["M"], int(result["hypermeter"]["D"])*int(result["hypermeter"]["M"]), result["time"]]

def display_results(results, experiments_params):
    sorted_results = {}
    for device in experiments_params["params"][3]["device"]:
        sorted_results[str(device)] = []
    max_offset = 10
    for result in results:
        d = int(result["hypermeter"]["D"])
        m = int(result["hypermeter"]["M"])
        dm = (d * m)<<10 + d
        device = result["hypermeter"]["device"]
        
        if len(sorted_results[device]) == 0:
            sorted_results[device].append(get_result(result))
            continue
        
        last_idx = 0
        for idx, sorted_result in enumerate(sorted_results[device]):
            sorted_d = int(sorted_result[0])
            sorted_m = int(sorted_result[1])
            sorted_dm = (sorted_d * sorted_m)<<max_offset + sorted_d
            
            last_sorted_d = int(sorted_results[device][last_idx][0])
            last_sorted_m = int(sorted_results[device][last_idx][1])
            last_sorted_dm = (last_sorted_d * last_sorted_m)<<max_offset + last_sorted_d
            
            if last_sorted_dm < dm and dm < sorted_dm:
                sorted_results[device].insert(idx,get_result(result))
                break
            elif dm < last_sorted_dm and idx == 0:
                sorted_results[device].insert(last_idx,get_result(result))
                break
            elif dm > sorted_dm and idx == len(sorted_results[device])-1:
                sorted_results[device].insert(idx+1,get_result(result))
            
            last_idx = idx
    # print(f"sorted result {sorted_results}")
    return sorted_results

# Step 1: Prepare parameters for batch evaluation

In this part, we set the parameters for batch evaluation

In [4]:
# parameters for experiments
experiment_name = "molecular-unfolding"
data_path = "molecular-unfolding-data"
experiments_params =  {
    "version": "1",
    "params": [
        {"M": [1,2,3,4,5,6]},
        {"D": [8]},
        {"shots": [10000]},
        {"device": [{"qc": "null", "cc": "ml.m5.large"},{"qc": "null", "cc": "ml.m5.4xlarge"}]}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

print(f"parameters for experiments: \n {hybrid_job_params}")

parameters for experiments: 
 [{'M': 1, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 1, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'M': 2, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 2, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'M': 3, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 3, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'M': 4, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 4, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'M': 5, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 5, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}, {'M': 6, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'M': 6, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.

In [5]:
# Upload dataset to S3
s3_path = upload_data(data_path)
print(f"upload data to s3 path: {s3_path}")

upload data to s3 path: s3://amazon-braket-us-east-1-002224604296/molecular-unfolding-data


# Step 2: Prepare image for experiment

In this part, we use the following code to prepare the image for experiment. For the first run, 
please run build_and_push.sh to create the image. For future experiments, avoid running
build_and_push.sh unless you want to rebuild the image

In [6]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.client('s3').meta.region_name
image_name = f"amazon-braket-{experiment_name}-jobs"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"

print(f"the hybrid job image for {account_id} in region {region}: {image_uri}")

!sh build_and_push.sh {image_name}

the hybrid job image for 002224604296 in region us-east-1: 002224604296.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-molecular-unfolding-jobs:latest


In [13]:
hybrid_jobs_json = f"{experiment_name}-hybrid-jobs.json"
print(f"job info will be saved in {hybrid_jobs_json}")

job info will be saved in molecular-unfolding-hybrid-jobs.json


# Step 3: Launch Amazon Braket Hybrid Jobs for experiment

In this part, we use the following code to launch the same number of hybrid jobs as the sets of parameters for this experiments.
When the number of jobs exceeds 5, this thread will wait

In [12]:
# Long runnning cell due to Burst rate of CreateJob requests < 5 RPS
# sudo apt-get install python-prctl at first
# https://stackoverflow.com/questions/34361035/python-thread-name-doesnt-show-up-on-ps-or-htop
from threading import Thread
import threading
import setproctitle

def launch_hybrid_jobs(hybrid_job_params=hybrid_job_params, hybrid_jobs_json=hybrid_jobs_json):
    setproctitle.setproctitle(threading.current_thread().name)
    # parse evaluation parameters and trigger hybrid jobs:
    jobs = []
    names = []

    job_name = f"{experiment_name}-job"

    # from braket.jobs.local import LocalQuantumJob

    for job_param in hybrid_job_params:
        M = job_param['M']
        D = job_param['D']
        quantum_device = get_quantum_device(job_param['device']['qc'])
        classical_device = job_param['device']['cc']

        device_name = classical_device.replace(".","-")
        device_name = device_name.replace("x","")
        
        name = f"{experiment_name}-M-{M}-D-{D}-{device_name}" + str(int(time.time()))

        tmp_job = AwsQuantumJob.create(
            device=quantum_device,
            source_module=f"{experiment_name}",
            entry_point=f"{experiment_name}.{job_name}:main",
            job_name=name,
            hyperparameters=job_param,
            input_data=s3_path,
            instance_config=InstanceConfig(instanceType=classical_device),
            image_uri=image_uri,
            wait_until_complete=False,
        )
        
        print(f"Finish create {experiment_name} with M {M}, D {D} and device {device_name}")

        jobs.append(tmp_job)
        names.append(name)


        while not queue_check(jobs):
            time.sleep(5)
    jobs_arn = []

    for job in jobs:
        jobs_arn.append(job.arn)

    jobs_states = {
        "experiment_name": experiment_name,
        "hybrid-jobs-arn": jobs_arn,
        "names": names
    }
    
    # save hybrid job arn for further analysis
    json_object = json.dumps(jobs_states, indent=4)

    with open(hybrid_jobs_json, "w") as outfile:
        outfile.write(json_object)
        
    print(f"Finish launch all the hybrid jobs and save all the files")

t = Thread(target=launch_hybrid_jobs, name="launch-hybrid-job", daemon=True).start()

fail to get null: list index out of range, use sv1 instead
Finish create molecular-unfolding with M 1, D 8 and device ml-m5-large
queue_count 1, running_count 0
fail to get null: list index out of range, use sv1 instead
Finish create molecular-unfolding with M 1, D 8 and device ml-m5-4large
queue_count 1, running_count 1
fail to get null: list index out of range, use sv1 instead
Finish create molecular-unfolding with M 2, D 8 and device ml-m5-large
queue_count 1, running_count 2
fail to get null: list index out of range, use sv1 instead
Finish create molecular-unfolding with M 2, D 8 and device ml-m5-4large
queue_count 1, running_count 3
fail to get null: list index out of range, use sv1 instead
Finish create molecular-unfolding with M 3, D 8 and device ml-m5-large
queue_count 1, running_count 4
queue_count 0, running_count 5
queue_count 0, running_count 5
queue_count 0, running_count 5
queue_count 0, running_count 5
queue_count 0, running_count 5
queue_count 0, running_count 5
queue_c

In [14]:
# run the following scripts to check the created threads
!ps -aux | grep launch-hybrid-job

ubuntu     85131  0.0  1.3 1667824 438060 ?      Ssl  Jan05   0:35 launch-hybrid-job
ubuntu     90748  0.0  1.5 1864064 491840 ?      Ssl  Jan06   1:49 launch-hybrid-job
ubuntu    138118  0.0  1.6 1754604 523720 ?      Ssl  Jan10   0:22 launch-hybrid-job
ubuntu    145298  0.0  1.6 1823156 526432 ?      Rsl  Jan11   0:18 launch-hybrid-job
ubuntu    157651  0.0  0.0   8748  3444 pts/0    Ss+  11:48   0:00 /bin/bash -c ps -aux | grep launch-hybrid-job
ubuntu    157653  0.0  0.0   8176   664 pts/0    S+   11:48   0:00 grep launch-hybrid-job


# Step 4: Jobs finish and visualize results

Please use the following code to check the status of hybrid jobs. The status of hybrid jobs can also be checked in the Amazon Braket console. Optionally, if the email if input when deploying the solution, emails will be sent at the same number of hybrid jobs once 
the status of jobs changes.

In [16]:
# run the following code to test whether all the jobs finish
results = []
if os.path.exists(hybrid_jobs_json):
    # recover hybrid jobs and show result
    jobs_states_load = None
    with open(hybrid_jobs_json, "r") as outfile:
        jobs_states_load = json.load(outfile)

    completed_jobs_arn = set()

    for job_name, job_arn in zip(jobs_states_load["names"], jobs_states_load["hybrid-jobs-arn"]):
        current_job = AwsQuantumJob(job_arn)
        print(f"the state of job {job_name} is : {current_job.state()}")
        if current_job.state() == 'COMPLETED':
            completed_jobs_arn.update({job_arn})

    whole_jobs_num = len(jobs_states_load["names"])

    results = None
    if len(completed_jobs_arn) == whole_jobs_num:
        print(f"all jobs completed")
        results = []
        for job_arn in completed_jobs_arn:
            current_job = AwsQuantumJob(job_arn)
            results.append(current_job.result())
        # display results
        results = display_results(results, experiments_params)
else:
    print(f"JSON file for job arns not generated! please wait for the thread(launch-hybrid-job) to finish")

the state of job molecular-unfolding-M-1-D-8-ml-m5-large1673608692 is : COMPLETED
the state of job molecular-unfolding-M-1-D-8-ml-m5-4large1673608697 is : COMPLETED
the state of job molecular-unfolding-M-2-D-8-ml-m5-large1673608702 is : COMPLETED
the state of job molecular-unfolding-M-2-D-8-ml-m5-4large1673608706 is : COMPLETED
the state of job molecular-unfolding-M-3-D-8-ml-m5-large1673608711 is : COMPLETED
the state of job molecular-unfolding-M-3-D-8-ml-m5-4large1673608843 is : COMPLETED
the state of job molecular-unfolding-M-4-D-8-ml-m5-large1673608856 is : COMPLETED
the state of job molecular-unfolding-M-4-D-8-ml-m5-4large1673608862 is : COMPLETED
the state of job molecular-unfolding-M-5-D-8-ml-m5-large1673608868 is : COMPLETED
the state of job molecular-unfolding-M-5-D-8-ml-m5-4large1673608942 is : COMPLETED
the state of job molecular-unfolding-M-6-D-8-ml-m5-large1673609027 is : COMPLETED
the state of job molecular-unfolding-M-6-D-8-ml-m5-4large1673609141 is : COMPLETED
all jobs c

In [17]:
rename_result = {}
device_list = []
x_list = []
y_list = []
for k,vs in results.items():
    k = k.replace("\'","\"")
    dict_k = json.loads(k)
    device_name = None
    if dict_k['qc'] == 'null':
        device_name = dict_k['cc']
    else:
        device_name = dict_k['qc']
    for v in vs:
        device_list.append(device_name)
        x_list.append(v[2])
        y_list.append(v[3])
source = pd.DataFrame({
    "DxM": np.array(x_list),
    "Time to Solution": np.array(y_list),
    "Device": np.array(device_list),
})

alt.Chart(source).mark_line(point = True).encode(
    x='DxM',
    y='Time to Solution',
    color='Device',
).properties(
    title = "Molecular unfolding experiments",
    width = 700,
    height = 600,
).interactive()

alt.Chart(...)